In [1]:
!pip install numpy plotly

tensorflow 1.9.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.0.0 which is incompatible.
spacy 2.0.12 has requirement regex==2017.4.5, but you'll have regex 2018.7.11 which is incompatible.
pypi-publisher 0.0.4 has requirement gitpython==0.3.6, but you'll have gitpython 2.1.11 which is incompatible.
pyfunctional 1.1.3 has requirement dill==0.2.5, but you'll have dill 0.2.8.2 which is incompatible.
pyfunctional 1.1.3 has requirement six==1.10.0, but you'll have six 1.11.0 which is incompatible.
flake8 3.5.0 has requirement pyflakes<1.7.0,>=1.5.0, but you'll have pyflakes 2.0.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from ipywidgets import interact_manual
from itertools import combinations
from math import log

import numpy as np
from plotly.graph_objs import *
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_splits(total_sales, total_tax, sales):
    sales_and_tax = np.vstack([sales, (np.asarray([5, 12, 18, 28]) / 100)])
    tax_rates_of_unk = sales_and_tax[1, np.isnan(sales)]
    sales_of_unk = total_sales - np.sum(sales[~np.isnan(sales)])
    tax_of_unk = total_tax - np.sum(np.prod(sales_and_tax[:, ~np.isnan(sales)], axis=0))
    alpha = (tax_of_unk - sales_of_unk * tax_rates_of_unk[1]) / (tax_rates_of_unk[0] - tax_rates_of_unk[1])
    beta = (tax_of_unk - sales_of_unk * tax_rates_of_unk[0]) / (tax_rates_of_unk[1] - tax_rates_of_unk[0])
    sales[np.isnan(sales)] = np.asarray([alpha, beta])
    if len(list(filter(lambda x: x, list(sales < 0)))) > 0: 
        raise ValueError("Invalid input values. No solution possible.")
    return sales

In [4]:
def get_valid_solutions(total_sales, total_tax, start, interval):
    valid_solutions = []
    for pos in combinations(list(range(4)), 2):
        X = np.linspace(start, total_sales, (total_sales / interval))
        Y = np.linspace(start, total_sales, (total_sales / interval))
        mesh = np.array(np.meshgrid(X, Y)).T.reshape(-1, 2)
        mesh_with_nan = np.full([mesh.shape[0], 4], np.nan)
        mesh_with_nan[:, pos] = mesh
        for vals in mesh_with_nan:
            try:
                valid_solutions.append(get_splits(total_sales, total_tax, vals))
            except ValueError:
                pass
    return np.asarray(valid_solutions)

In [5]:
def plot_solutions(total_sales, total_tax):
    start = 0
    interval = 10 ** (int(log(total_sales, 10) - 1))
    solutions = get_valid_solutions(total_sales, total_tax, start, interval)
    if solutions.size == 0:
        print("Invalid input values. No solution Possible.")
        return
    texts = list(map(lambda x: f"""@5%: {int(x[0])}</br></br>@12%: {int(x[1])}</br>@18%: {int(x[2])}</br>@28%: {int(x[3])}""", solutions))
    data = [
        Scatter3d(
            x=solutions[:,0],
            y=solutions[:, 1],
            z=solutions[:, 2],
            mode='markers',
            marker=dict(
                size=4,
                color=solutions[:, 3],
                colorscale='Viridis',   # choose a colorscale
            ),
            text=texts,
            hoverinfo = 'text'
        )
    ]
    layout = Layout(
        title=f'Possible Solutions for Total Sales {total_sales} and Total Tax {total_tax}',
        scene = dict(
            xaxis=dict(
                title='Sales @5%'
            ),
            yaxis=dict(
                title='Sales @12%'
            ),
            zaxis=dict(
                title='Sales @18%'
            )
        )
    )
    fig = Figure(data=data, layout=layout)
    iplot(fig)

In [6]:
interact_manual(plot_solutions,
         total_sales=(1000, 100000, 10000),
         total_tax=(50, 18000, 500))

interactive(children=(IntSlider(value=41000, description='total_sales', max=100000, min=1000, step=10000), Int…

<function __main__.plot_solutions(total_sales, total_tax)>

In [7]:
plot_solutions(total_sales=30000, total_tax=2500)